Helpful to know exists - Request body:   
https://platform.openai.com/docs/api-reference/responses/create
- parallel tool calling: allow calling tools in parallel or not
- previous_response_id: allows for multi-turn
  - instructions: will not be carried over to next call if multi-turn activated
- include: can include files returned in search after tool call
- tool_choice: how the model should select which tool to choose when generating a response
- tools: the actual tools to call (web search, file search)

- gpt 4.1 nano is most cost effective for now

#### Import Credentials

In [ ]:
# !pip install openai
# !pip install requests

In [22]:
from creds import credentials
import weather
import json
from openai import OpenAI
import os

In [ ]:
# get keys
openai_key = credentials['tool-calling-OpenAI']
weather_api = credentials['weather_api']

#### Make tool
Must define function to use + how to use it

In [13]:
weather_tool = [
    # weather tool
    {
        "type": "function",
        "name": "get_weather",
        "description":"get temperature, humidity, precipitation, wind speed for specified location for the next 5 days.",
        "parameters":{
            "type": "object",
            "properties": {
                "lat": {"type":"number"},
                "long":{"type":"number"}
            },
            "required": ["lat", "long"],
            "additionalProperties": False
        },
    "strict" : True
}]

In [7]:
input_message = [{
    "role":"user",
    "content": "What is the weather going to be in Detroit?"
}]

#### Agent #1: Extract location and get weather from prompt
Get next 5 days of weather

In [ ]:
client = OpenAI(api_key=openai_key)

response = client.responses.create(
    model="gpt-4.1-nano",
    input=input_message,
    tools=weather_tool
)

print(response.output_text)


In [ ]:
tool_call = response.output[0]
args = json.loads(tool_call.arguments)

result = weather.get_weather(args["lat"], args["long"])

#### Agent #2: Review the weather and evaluate when it would be best to run outside

In [27]:
result.text

'{"data":{"timelines":[{"timestep":"1d","endTime":"2025-04-23T10:00:00Z","startTime":"2025-04-18T10:00:00Z","intervals":[{"startTime":"2025-04-18T10:00:00Z","values":{"humidityAvg":54,"precipitationProbability":0,"temperature":80.6,"windSpeed":16.9}},{"startTime":"2025-04-19T10:00:00Z","values":{"humidityAvg":68,"precipitationProbability":0,"temperature":65,"windSpeed":14.7}},{"startTime":"2025-04-20T10:00:00Z","values":{"humidityAvg":68,"precipitationProbability":20,"temperature":52,"windSpeed":11.2}},{"startTime":"2025-04-21T10:00:00Z","values":{"humidityAvg":74,"precipitationProbability":35,"temperature":72.2,"windSpeed":18.1}},{"startTime":"2025-04-22T10:00:00Z","values":{"humidityAvg":73,"precipitationProbability":0,"temperature":60.8,"windSpeed":11.3}},{"startTime":"2025-04-23T10:00:00Z","values":{"humidityAvg":88,"precipitationProbability":10,"temperature":67.5,"windSpeed":8.1}}]}]}}'